# Notebook 03: Modeling
#### Purpose:
The purpose of this notebook is to train a model to semantically segment aerial cityscape imagery.

#### Data:
The training data comes from the Varied Drone Dataset (VDD) and the Semantic Drone Dataset (SDD).

#### Process:
The modeling process that takes place is as follows:
1. 

### Imports

In [10]:
import os
import glob
import cv2
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

from keras.utils import to_categorical, normalize
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Conv2DTranspose, Dropout
from keras.callbacks import Callback, CSVLogger, EarlyStopping

### Functions

In [9]:
def get_image_data(folder, extension, image_shape):
    """
    Description
    -----------
    Loads images from folder into a single array.
    
    Parameters
    ----------
    folder : str
        the folder path (including trailing /).
    extension : str
        the file extension (i.g. .jpeg, .bmp).
    image_shape : tuple of int
        image height by image width in pixels.

    Returns
    -------
    image_list : numpy array
        loaded images stored in a numpy array.
    """
    image_list = []
    for directory_path in glob.glob(folder):
        for img_path in glob.glob(os.path.join(directory_path, "*" + extension)):
            img = cv2.imread(img_path, 0)  #being read as grayscale even though RGB
            img = cv2.resize(img, image_shape, interpolation=cv2.INTER_NEAREST)  # nearest neighbor interpolation to prevent creation of new mask values
            image_list.append(img)
           
    #Convert list to array for machine learning processing        
    return np.array(image_list)


def load_image_and_mask_data(image_folder, image_extension, mask_folder, mask_extension, image_shape):
    #Load and resize train images and masks
    images = get_image_data(image_folder, image_extension, image_shape)
    masks = get_image_data(mask_folder, mask_extension, image_shape)
    
    #Encode mask labels (enforces 0-indexed labeling)
    labelencoder = LabelEncoder()
    n, h, w = masks.shape
    masks_1_dim = masks.reshape(-1,1).ravel()
    masks_1_dim_encoded = labelencoder.fit_transform(masks_1_dim)
    masks_encoded = masks_1_dim_encoded.reshape(n, h, w)
    
    #Expand image dimensions and normalize
    images_input = np.expand_dims(images, axis=3)  #(image, y, x, new_dimension)
    images_input = normalize(images_input, axis=1)  #normalize each image
    masks_expanded = np.expand_dims(masks_encoded, axis=3)  #(image, y, x, new_dimension)
    
    #Info
    class_values = np.unique(masks_expanded)
    n_classes = len(class_values)
    print("Class labels in the dataset are ... ", class_values)
    print("Number of classes in the dataset is ... ", n_classes)
    
    #One-hot encode mask classes
    masks_input = to_categorical(masks_expanded, num_classes=n_classes)
    
    #Calculate balanced class weights
    class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                     classes=class_values,
                                                     y=masks_1_dim_encoded)
    class_weights_dict = {}
    for ind, c in enumerate(np.unique(masks_1_dim_encoded)):
        class_weights_dict[c] = class_weights[c]
    print("Class weights dictionary looks like ...:", class_weights_dict)
    
    return images_input, masks_input, n_classes, class_weights_dict


def get_fold_indices(n_samples, n_folds):
    """
    Description
    -----------
    Splits n_samples indices into n_folds folds for cross-validation.
    
    Parameters
    ----------
    n_samples : int
        number of images in the training set to be used in the cross-validation.
    n_folds : int
        number of folds to split the training data into for cross-validation.

    Returns
    -------
    indices_by_fold : dict
        dictionary of image indices split into `n_folds` folds (each fold gets its own key).
    """
    x = n_samples // n_folds
    
    indices_by_fold = {}
    
    for n in range(n_folds):
        fold_n = [i for i in range(x*n, x*n+x)]
        
        if n == (n_folds-1):
            fold_n.extend([i for i in range(max(fold_n)+1,n_samples)])
        
        indices_by_fold[n] = fold_n
    
    return indices_by_fold


def get_train_val_fold_split(indices_by_fold, current_fold):
    """
    Description
    -----------
    For the current_fold, creates an array of training indices and an array of validation indices.
    
    Parameters
    ----------
    indices_by_fold : dict
        dict returned from get_fold_indices.
    current_fold : int
        zero-based fold number in range of cross-validation folds.

    Returns
    -------
    i_train : list
        indices of images to be used in the training set (n-1 fold indices).
    i_val : list
        indices of images to be used in the validation set (1 fold indices).
    """
    i_train = []
    i_val = []
    for k in indices_by_fold.keys():
        if k == current_fold:
            i_val.extend(indices_by_fold[k])
        else:
            i_train.extend(indices_by_fold[k])
    return i_train, i_val

### Model(s)

In [1]:
def multi_class_unet_model(n_classes=4, IMG_HEIGHT=256, IMG_WIDTH=256, IMG_CHANNELS=1):
    #Build the model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    s = inputs

    #Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
     
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
     
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
     
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    
    #Expansive path 
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
     
    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
     
    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
     
    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
    
    #Multi-class activation
    outputs = Conv2D(n_classes, (1, 1), activation='softmax')(c9)
    
    #Create model
    model = Model(inputs=[inputs], outputs=[outputs])
    
    return model

### Custom Callbacks

In [9]:
class CustomSaver_BatchSizeStudy(Callback):
    def __init__(self, study_folder, epochs_list, batch_size, fold):
        self.study_folder = study_folder
        self.epochs_list = epochs_list
        self.batch_size = batch_size
        self.fold = fold
        super().__init__()
        
    def on_epoch_end(self, epoch, logs={}):
        if epoch in epochs:  # or save after some epoch, each k-th epoch etc.
            self.model.save(self.study_folder + "models/model_bs_{}_f_{}_e_{}.keras".format(self.batch_size, self.fold, self.epochs_list[self.epochs_list.index(epoch)]))
            print("model saved ...")

class CustomSaver_LearningRateStudy(Callback):
    def __init__(self, study_folder, epochs_list, learning_rate, fold):
        self.study_folder = study_folder
        self.epochs_list = epochs_list
        self.learning_rate = learning_rate
        self.fold = fold
        super().__init__()
        
    def on_epoch_end(self, epoch, logs={}):
        if epoch in epochs:  # or save after some epoch, each k-th epoch etc.
            self.model.save(self.study_folder + "models/model_l_{}_f_{}_e_{}.keras".format(str(self.learning_rate).split('.')[1], self.fold, self.epochs_list[self.epochs_list.index(epoch)]))
            print("model saved ...")

### Common Inputs

In [2]:
#cross-validation
folds = 3

#image size
SIZE_X = 512
SIZE_Y = 512

#filepaths
image_folder = "../data/Master/src/"
image_extension = ".jpeg"

mask_folder = "../data/Master/gt/"
mask_extension = ".bmp"

### Batch Size and Epoch Study

In [10]:
batch_size = [8, 16, 32, 64]
epochs = [10, 50, 100]  #continuously runs to largest epoch in array, but saves model at each epoch in array
study_folder = '../batch_size_epoch_study/'

#create directories
study_dir = os.path.dirname(study_folder)
if not os.path.isdir(study_dir):
    os.makedirs(study_dir)
    os.makedirs(study_dir+'/logs/')
    os.makedirs(study_dir+'/models/')

Class labels in the dataset are ...  [0 1 2 3 4 5 6 7]
Number of classes in the dataset is ...  8
Class weights dictionary looks like ...: {0: 0.36735640506122164, 1: 4.171348015131236, 2: 1.1228593437742438, 3: 0.45889725780832913, 4: 2.0337580794243313, 5: 17.164363482076368, 6: 0.7339905890630258, 7: 17.85055714896619}


In [ ]:
#load data
image_input, mask_input, n_classes, class_weights_dict = load_image_and_mask_data(image_folder, image_extension,
                                                                                  mask_folder, mask_extension,
                                                                                  (SIZE_Y, SIZE_X))

In [ ]:
#instantiate and compile model
model = multi_class_unet_model(n_classes=n_classes, IMG_HEIGHT=SIZE_Y, IMG_WIDTH=SIZE_X, IMG_CHANNELS=1)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#get cross-validation indices dictionary
indices_by_fold = get_fold_indices(X_train.shape[0], folds)

In [ ]:
#train model
for b in batch_size:
    for f in range(folds):
        print(f"Training model with batch size: {b}, fold: {f}")
        #get cross-validation indices
        i_train, i_val = get_train_val_fold_split(indices_by_fold, f)

        #callbacks
        csv_logger = CSVLogger(study_folder + f'logs/B{b}_F{f}.log', separator=',', append=False)
        early_stopping = EarlyStopping(monitor='val_loss', patience=3, start_from_epoch=10)
        saver = CustomSaver_BatchSizeStudy(study_folder, epochs, b, f)

        #train
        history = model.fit(image_input[i_train], mask_input[i_train],
                            batch_size=b,
                            verbose=2,
                            epochs=e,
                            validation_data=(image_input[i_val], mask_input[i_val]),
                            class_weight=class_weights_dict,
                            shuffle=False,
                            callbacks=[csv_logger, early_stopping, saver])

### Learning Rate Study

In [ ]:
learning_rates = [0.0005, 0.001, 0.01, 0.1, 0.2, 0.3]  #default learning rate is 0.001
study_folder = '../learning_rate_study/'

#create directories
study_dir = os.path.dirname(study_folder)
if not os.path.isdir(study_dir):
    os.makedirs(study_dir)
    os.makedirs(study_dir+'/logs/')
    os.makedirs(study_dir+'/models/')

In [ ]:
#train model
for l in learning_rates:
    #instantiate and compile model with modified optimizer
    model = multi_class_unet_model(n_classes=n_classes, IMG_HEIGHT=SIZE_Y, IMG_WIDTH=SIZE_X, IMG_CHANNELS=1)
    opt = Adam(learning_rate=l)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    for f in range(folds)
        print(f"Training model with learning rate: {l}, fold: {f}")
        #get cross-validation indices
        i_train, i_val = get_train_val_fold_split(indices_by_fold, f)
        
        #callbacks
        csv_logger = CSVLogger(study_folder + f"logs/L{str(l).split('.')[1]}_F{f}.log", separator=',', append=False)
        early_stopping = EarlyStopping(monitor='val_loss', patience=3, start_from_epoch=10)
        saver = CustomSaver_LearningRateStudy(study_folder epochs, l, f)

        #train
        history = model.fit(image_input[i_train], mask_input[i_train],
                            batch_size=8,
                            verbose=2,
                            epochs=50,
                            validation_data=(image_input[i_val], mask_input[i_val]),
                            class_weight=class_weights_dict,
                            shuffle=False,
                            callbacks=[csv_logger, early_stopping, saver])

### Review Batch Size and Epoch Study Results

### Review Learning Rate Study Results